In [1]:
import flax
import glob
import h5py
import jax


import matplotlib.pyplot as plt
import numpy as np
import os
import pyscf

from flax import linen as nn
from glob import glob
from jax import numpy as jnp

from pyscf import gto, scf
from pyscf.dft import numint


from typing import Callable, Sequence, Optional, NamedTuple

In [ ]:
class Grid(NamedTuple):
    coords : Array
    weights : Array

def integrate(grid: Grid, vals = Array, axes=(0,axis):
    return jnp.tensordot(grid.weights,vals, axes=(0,axis))

In [2]:


def load_file(path):
    all_coords = []
    all_weights = []
    all_rho = []
    all_GradR = []
    all_lda = []
    all_pbe = []
    
    with h5py.File(path, 'r') as file:
        num_entries = len(file['coords'])  # Assuming all datasets have the same length
        
        for x in range(num_entries):
            all_coords.append(np.array(file['coords'][x]))
            all_weights.append(np.array(file['weights'][x]))
            all_rho.append(np.array(file['rho'][x]))
            all_GradR.append(np.array(file['grad_rho'][x]))
            all_lda.append(np.array(file['exc_lda'][x]))
            all_pbe.append(np.array(file['exc_pbe'][x]))
    
    return [load_file(data)]

def load_data(folder_path):
    paths = glob.glob(folder_path + '/*.h5')
    return [load_file(path) for path in paths]

def separate_data(data):
    return zip(*data)

# Example usage:
data_folder = "/Users/corneliussalonis/Research/SNSP_23/MLtut/cornelius_data"
data = load_data(data_folder)
separated_data = separate_data(data)


In [3]:
len(data)

19

In [4]:
def load_file(path):
    with h5py.File(path,'r') as file:
        for x in len(path):
            all_coords[x] = np.array(file['coords'])
            all_weights[x]= np.array(file['weights'])
            all_rho[x] = np.array(file['rho'])
            all_GradR[x] = np.array(file['grad_rho'])
            all_lda[x] = np.array(file['exc_lda'])
            all_pbe[x]= np.array(file['exc_pbe'])
            #grid = Grid(all_coords,all_weights)
            #inputs = FunctionalInputs(grid,all_rho, all_GradR)
    
def load_data(folder_path):
    paths = glob(folder_path)
    return [load_file(path) for path in paths]

def seperate_data(data):
    return zip(*data)


In [5]:
data1 = load_data('/Users/corneliussalonis/Research/SNSP_23/MLtut/cornelius_data/*-gauss_chebyshev*')
data1

TypeError: 'module' object is not callable

In [ ]:
a = 0 
for x in range(len(path)):
    with h5py.File(path[x], 'r') as file:
    #print(file.keys())
        all_coords[f'Coords of {path[x]}'] = np.array(file['coords'])
        trial_weights = np.array(file['weights'])

        trial_rho = np.array(file['rho'])
    #trial_rho = jnp.broadcast_to(trial_rho / 2, (2, *trial_rho.shape))

        Tgrad_rho = np.array(file['grad_rho'])
    #Tgrad_rho = jnp.broadcast_to(Tgrad_rho, (2, *Tgrad_rho.shape))

        exc_lda = np.array(file['exc_lda'])
        exc_pbe = np.array(file['exc_pbe'])


    
    

In [ ]:
all_coords

In [ ]:
len(trial_weights)

In [ ]:
#fixed_rho = jnp.broadcast_to(trial_rho / 2, (2, *trial_rho.shape))
#fixed_grho = jnp.broadcast_to(Tgrad_rho, (2, *Tgrad_rho.shape))
#norm2 = jnp.sum(Tgrad_rho**2, axis = -1)
#spin2 = jnp.sum(Tgrad_rho.sum(axis = 0, keepdims = True)**2, axis = -1)
#feature_list = [trial_rho, Tgrad_rho,norm2,spin2]
#print(fixed_grho.shape,fixed_rho.shape)
#new = jnp.concatenate(feature_list, axis=0).transpose()

In [ ]:
Array = jax.Array

In [ ]:
class Grid(NamedTuple):
    coords : Array
    weights : Array

def integrate(grid: Grid, vals = Array, axis: int = 0)-> Array:
    return jnp.tensordot(grid.weights, vals, axes=(0,axis))

In [ ]:
class FunctionalInputs(NamedTuple):
    grid: Grid
    rho: Array
    grad_rho: Optional[Array] = None

In [ ]:
def build_input_array(inputs: FunctionalInputs):
    _, rho, grad_rho = inputs

    if grad_rho is None:
        feature_list = [rho]

    else:
        #grad_rho = grad_rho.transpose()
        grad_rho_norm = jnp.sum(grad_rho**2, axis=-1)
        feature_list = [rho,grad_rho_norm]
    return jnp.stack(feature_list, axis = -1)

In [ ]:
class ForwardFeedNN(nn.Module):
    
    layer_widths: Sequence[int]
    out_features = 1
    activate : Callable[[Array],Array] = jax.nn.gelu
    squash_offset: float = 1e-4

    @nn.compact
    def __call__(self,inputs: FunctionalInputs):
        
        x = build_input_array(inputs)

        h = jnp.log(jnp.abs(x)+self.squash_offset)
        h = nn.Dense(features = self.layer_widths[0])(h)
        h = jnp.tanh(h)

        for feature in self.layer_widths:
            h = nn.Dense(features = feature)(h)
            h = nn.LayerNorm()(h)
            h = self.activate(h)
            
        h = nn.Dense(features = self.out_features)(h)
        return 2 * jax.nn.sigmoid(h)


In [ ]:
grid = Grid(coords = trial_coords, weights = trial_weights)
inputs = FunctionalInputs(grid, rho = trial_rho, grad_rho = Tgrad_rho)

#fxc = ForwardFeedNN(layer_widths= [256,256,256])

#key = jax.random.PRNGKey(16)
#params = fxc.init(key,inputs)
grid.coords.shape
    

In [ ]:
inputs.rho.shape, inputs.grad_rho.shape

In [ ]:
x = build_input_array(inputs)

In [ ]:
x.shape

In [ ]:
fxc = ForwardFeedNN(layer_widths=(128,128))
key = jax.random.PRNGKey(42)
params = fxc.init(key, inputs)

In [ ]:
exc = fxc.apply(params,inputs)
#exc.shape
(grid.weights.shape, grid.coords.shape, trial_rho.shape, exc.shape)

In [ ]:
integrate(grid, trial_rho * exc.squeeze())

In [ ]:
exc_lda.shape

In [ ]:
plt.grid()
plt.xlabel('Distance from Nucleus')
plt.ylabel('Exchange Correlation Energy')
plt.scatter(trial_coords,exc_lda)
plt.scatter(trial_coords,exc, c = 'r')
plt.legend()

In [ ]:
Dexc = jnp.sum(exc_lda)
Dexc

In [ ]:
# def cs_fun(params, Texc):
#     Pexc = integrate(grid, trial_rho * exc.squeeze())
#     return jnp.mean((Texc-Pexc)**2)

# cs_fun(params, exc_lda)

In [ ]:
@jax.jit
def cost_one_input(params, inputs, target_exc):
    exc = fxc.apply(params, inputs).squeeze()
    return integrate(inputs.grid, (exc - target_exc)**2)

@jax.value_and_grad
def cost_batch(params, inputs: Sequence[FunctionalInputs], target_excs: Sequence[Array]):

    batch_size = len(inputs)
    mean_cost = 0.0

    for input, target in zip(inputs, target_excs):
        mean_cost += cost_one_input(params, input, target) / batch_size

    return mean_cost

### The cost function is integrated mean squared error between the predicted and true values of the output

$$ \texttt{cost} = \int d^3 x \; (\epsilon ^{NN} _{xc} - \epsilon ^{\text{target}} _{xc}) ^ 2$$

averaged over the batch

In [ ]:
def load_file(path):

    with h5py.File(path, 'r') as file:

        coords = np.array(file['coords'])
        weights = np.array(file['weights'])

        rho = np.array(file['rho'])

        grad_rho = np.array(file['grad_rho'])

        exc = np.array(file['exc_lda'])

    grid = Grid(coords, weights)
    inputs = FunctionalInputs(grid, rho, grad_rho)

    return inputs, exc

def load_data(folder_path):
    paths = glob(folder_path + '/*.h5')
    return [load_file(path) for path in paths]

def separate_data(data):

    # data is a list of tuples
    # this trick separates the inputs and exc (targets)
    # example: inputs, targets = separate_data(data)

    return zip(*data)

In [ ]:
data = load_data('./cornelius_data/')

In [ ]:
len(data) # loaded 19 files

In [ ]:
# Super important - train-test split
train_data = data[:16]
test_data = data[16:]

In [ ]:
def divide_into_batches(data, batch_size):
    return [data[i:i+batch_size] for i in range(0, len(data), batch_size)]

In [ ]:
batches = divide_into_batches(train_data, 4)

In [ ]:
len(batches)

In [ ]:
test_inputs, test_targets = separate_data(batches[0])

In [ ]:
cost_one_input(params, test_inputs[0], test_targets[0])

In [ ]:
value, grad = cost_batch(params, test_inputs, test_targets)

In [ ]:
value

In [ ]:
grad

In [ ]:
grad_cost = jax.value_and_grad(cs_fun)

In [ ]:
grad_cost

In [ ]:
grad_cost(params, Dexc)

In [ ]:
def update_param(params,Dexc,lr = .001):
    cost, grad = grad_cost(params,Dexc)
    params = params - (lr * grad)
    return params, cost

In [ ]:
costs = []
for n in range(1000):
    params, cost = update_param(params,Dexc)
    costs.append(cost)